In [1]:
import pprint as pp
import re
import ipaddress
from mac_vendor_lookup import MacLookup

ipinfo = {}

fields = {'VM':None, 'Host':None, 'OS':None,
          'Tools':None, 'NicType':None, 'VLAN':None,
          'IP':None, 'Gateway':None, 'Subnet Mask':None, 'IP range':None,
          'DNS':None,'MAC':None}

header = ['VM', 'Host', 'OS', 'Tools', 'NicType', 'VLAN',
          'IP', 'Gateway', 'Subnet Mask', 'IP range', 'DNS','MAC']


filename = 'LT_VMs_Network.txt'
output = 'output_LT.txt'

def cleandup(ip):
    iplist = ip.split(',')
    return ','.join(list(dict.fromkeys(iplist)))


def getrange(ip, mask):
    try:
        ip = ip.split(',', 1)[0].strip()
    except:
        pass
    
    subnet = ipaddress.ip_network((ip, mask), strict = False)
    return str(subnet.network_address) + ' - ' +  str(subnet.broadcast_address)


with open(filename, encoding='utf8') as file, open(output, 'w', encoding='utf8') as output:
    output.write('|'.join(header))
    output.write('\n')
    
    for i in file:
        line = i.strip().split(':', 1)

        try:
            key = line[0].strip()
            value = line[1].strip()

            if key != 'MAC':
                if key == 'Gateway' or 'IP':
                    value = cleandup(value)
                fields[key] = value

            else:
                fields[key] = value
                fields['IP range'] = getrange(fields['IP'], fields['Subnet Mask'])
                csv_string = ''
                for m in header:#output in ordered form
                    csv_string = csv_string + fields[m] + '|'
                csv_string = csv_string + '\n'                
                output.write(csv_string)
                #print(csv_string)

        except:
            #print('fuck...')
            pass

In [2]:
def summarize_ip(ipfile):
    iplist = []
    with open(ipfile, encoding='utf8') as f:
        for line in f:
            iplist.append(ipaddress.ip_address(line.strip()))
            
    ip_min = min(iplist)
    ip_max = max(iplist)
    
    print('MIN:', ip_min, 'MAX:', ip_max, '\n')
    for m in sorted(iplist):
        print(m)
    print()
    
    ip_range = ipaddress.summarize_address_range(ip_min, ip_max)
    for i in ip_range:
        #print(i, list(i.hosts()))
        print(i, ',', i.network_address, '-', i.broadcast_address)

In [3]:
def read_ip(ipfile):
    iplist = {}
    with open(ipfile, encoding='utf8') as f:
        for line in f:
            pair = line.split(',')
            key = pair[0].strip()
            value = pair[1].strip()
            iplist[key]= value
            #iplist.append(ipaddress.ip_address(line.strip()))
    return iplist

In [9]:
iplist = read_ip('iplist.txt')
ipnetwork = '10.5.96.1/24'
offset = 3

subnets = list(ipaddress.ip_network(ipnetwork, strict = False).subnets(offset))
for net in subnets:
    print(net, str(net.network_address), '-', str(net.broadcast_address), '\n')
    for ip in iplist.keys():
        if ipaddress.ip_address(ip) in net:
            print(ip, iplist[ip])
    print()

10.5.96.0/27 10.5.96.0 - 10.5.96.31 

10.5.96.25 Yes
10.5.96.26 Yes

10.5.96.32/27 10.5.96.32 - 10.5.96.63 

10.5.96.34 Yes
10.5.96.35 Yes
10.5.96.36 Yes
10.5.96.37 Yes
10.5.96.44 Yes
10.5.96.47 Yes
10.5.96.33 No

10.5.96.64/27 10.5.96.64 - 10.5.96.95 

10.5.96.77 Yes
10.5.96.82 Yes
10.5.96.83 Yes
10.5.96.87 Yes
10.5.96.88 Yes
10.5.96.91 Yes
10.5.96.93 Yes
10.5.96.94 Yes
10.5.96.95 Yes
10.5.96.76 No
10.5.96.85 No

10.5.96.96/27 10.5.96.96 - 10.5.96.127 

10.5.96.102 Yes
10.5.96.105 Yes
10.5.96.110 Yes
10.5.96.97 Yes
10.5.96.99 Yes
10.5.96.103 No
10.5.96.104 No
10.5.96.117 No
10.5.96.118 No
10.5.96.123 No
10.5.96.96 No

10.5.96.128/27 10.5.96.128 - 10.5.96.159 

10.5.96.130 Yes
10.5.96.128 No
10.5.96.140 No
10.5.96.141 No
10.5.96.142 No
10.5.96.143 No
10.5.96.144 No
10.5.96.145 No
10.5.96.147 No

10.5.96.160/27 10.5.96.160 - 10.5.96.191 


10.5.96.192/27 10.5.96.192 - 10.5.96.223 

10.5.96.206 Yes
10.5.96.210 Yes
10.5.96.212 Yes
10.5.96.214 Yes
10.5.96.222 No

10.5.96.224/27 10.5.96.224